In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PowerTransformer, LabelEncoder
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV, train_test_split
from sklearn.metrics import f1_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from category_encoders import TargetEncoder

# Load Data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Feature Engineering
def haversine_distance(lat1, lon1, lat2, lon2):
    # Convert degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    # Haversine formula
    dlat = lat2 - lat1 
    dlon = lon2 - lon1 
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6371 * c  # Earth's radius in kilometers
    return km

# Feature engineering for both train and test sets
for df in [train_df, test_df]:
    df['trans_datetime'] = pd.to_datetime(df['trans_date'] + ' ' + df['trans_time'])
    df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
    df['age'] = (df['trans_datetime'] - df['dob']).dt.days / 365.25
    df['hour'] = df['trans_datetime'].dt.hour
    df['day'] = df['trans_datetime'].dt.day
    df['month'] = df['trans_datetime'].dt.month
    df['weekday'] = df['trans_datetime'].dt.weekday
    df['distance'] = haversine_distance(df['lat'], df['long'], df['merch_lat'], df['merch_long'])
    df['amt_mean_user'] = df.groupby('cc_num')['amt'].transform('mean')
    df['amt_std_user'] = df.groupby('cc_num')['amt'].transform('std')
    df['amt_ratio'] = df['amt'] / df['amt_mean_user']
    df['amt_log'] = np.log1p(df['amt'])
    df['distance_log'] = np.log1p(df['distance'])
    df['hour_day_interaction'] = df['hour'] * df['day']
    df['amt_age_interaction'] = df['amt'] * df['age']
    df['amt_distance_interaction'] = df['amt'] * df['distance']

# Encode categorical variables
gender_map = {'F': 0, 'M': 1}
for df in [train_df, test_df]:
    df['gender'] = df['gender'].map(gender_map)

categorical_features = ['category', 'job', 'state']
encoder = TargetEncoder(cols=categorical_features)
encoder.fit(train_df[categorical_features], train_df['is_fraud'])
train_df[categorical_features] = encoder.transform(train_df[categorical_features])
test_df[categorical_features] = encoder.transform(test_df[categorical_features])

# Select features for modeling
features = [
    'amt', 'amt_mean_user', 'amt_std_user', 'amt_ratio', 'gender', 'category', 'age', 'city_pop', 
    'job', 'hour', 'day', 'month', 'weekday', 'distance', 'state', 'amt_log', 'distance_log',
    'hour_day_interaction', 'amt_age_interaction', 'amt_distance_interaction'
]

X = train_df[features]
y = train_df['is_fraud']
X_test = test_df[features]

# Handle missing values
imputer = SimpleImputer(strategy='median')
X = pd.DataFrame(imputer.fit_transform(X), columns=features)
X_test = pd.DataFrame(imputer.transform(X_test), columns=features)

# Normalize the data using PowerTransformer
scaler = PowerTransformer(method='yeo-johnson')
X = pd.DataFrame(scaler.fit_transform(X), columns=features)
X_test = pd.DataFrame(scaler.transform(X_test), columns=features)

# Address class imbalance using SMOTE
smote = SMOTE(random_state=42)
X, y = smote.fit_resample(X, y)

# Split the training data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# XGBoost Hyperparameter Tuning
param_dist = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'subsample': [0.6, 0.8, 1],
    'colsample_bytree': [0.6, 0.8, 1],
    'gamma': [0, 0.1, 0.3, 0.5],
    'reg_alpha': [0, 0.1, 0.5, 1],
    'reg_lambda': [1, 1.5, 2, 3]
}

model = xgb.XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=1
)

random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=50, scoring='f1', n_jobs=-1, cv=5, verbose=2, random_state=42)
random_search.fit(X, y)

# Use the best model found
best_model = random_search.best_estimator_

# Evaluate using F1-score and AUC
y_pred = best_model.predict(X_val)
f1 = f1_score(y_val, y_pred)
y_pred_proba = best_model.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred_proba)
print('Validation F1 Score after Tuning:', f1)
print('Validation ROC AUC Score:', roc_auc)

# Make predictions on the test set
test_preds = best_model.predict(X_test)

# Prepare the submission file
submission = pd.DataFrame({
    'id': test_df['id'],
    'is_fraud': test_preds
})

submission.to_csv('submission.csv', index=False)


Fitting 5 folds for each of 50 candidates, totalling 250 fits


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:57:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:57:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:57:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:57:51] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packa

[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_depth=5, n_estimators=300, reg_alpha=1, reg_lambda=2, subsample=1; total time=  13.8s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_depth=5, n_estimators=300, reg_alpha=1, reg_lambda=2, subsample=1; total time=  14.0s
[CV] END colsample_bytree=0.6, gamma=0, learning_rate=0.1, max_depth=5, n_estimators=300, reg_alpha=1, reg_lambda=2, subsample=1; total time=  14.1s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:58:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:58:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [15:58:05] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=300, reg_alpha=0, reg_lambda=1.5, subsample=0.6; total time=  17.4s
[CV] END colsample_bytree=1, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=300, reg_alpha=0, reg_lambda=1.5, subsample=0.6; total time=  17.4s
[CV] END colsample_bytree=1, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=300, reg_alpha=0, reg_lambda=1.5, subsample=0.6; total time=  17.4s
[CV] END colsample_bytree=1, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=300, reg_alpha=0, reg_lambda=1.5, subsample=0.6; total time=  17.5s
[CV] END colsample_bytree=1, gamma=0.1, learning_rate=0.01, max_depth=5, n_estimators=300, reg_alpha=0, reg_lambda=1.5, subsample=0.6; total time=  17.5s


...

[CV] END colsample_bytree=1, gamma=0, learning_rate=0.1, max_depth=3, n_estimators=300, reg_alpha=1, reg_lambda=3, subsample=0.6; total time=  17.7s
[CV] END colsample_bytree=1, gamma=0, learning_rate=0.1, max_depth=3, n_estimators=300, reg_alpha=1, reg_lambda=3, subsample=0.6; total time=  17.5s
[CV] END colsample_bytree=1, gamma=0, learning_rate=0.1, max_depth=3, n_estimators=300, reg_alpha=1, reg_lambda=3, subsample=0.6; total time=  17.2s


/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:04:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:04:56] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[CV] END colsample_bytree=1, gamma=0, learning_rate=0.1, max_depth=3, n_estimators=300, reg_alpha=1, reg_lambda=3, subsample=0.6; total time=  17.0s
[CV] END colsample_bytree=1, gamma=0, learning_rate=0.1, max_depth=3, n_estimators=300, reg_alpha=1, reg_lambda=3, subsample=0.6; total time=  16.7s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=5, n_estimators=300, reg_alpha=0.5, reg_lambda=2, subsample=1; total time=  17.7s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=5, n_estimators=300, reg_alpha=0.5, reg_lambda=2, subsample=1; total time=  18.0s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=5, n_estimators=300, reg_alpha=0.5, reg_lambda=2, subsample=1; total time=  16.4s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=5, n_estimators=300, reg_alpha=0.5, reg_lambda=2, subsample=1; total time=  11.7s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.2, max_depth=5, n_estimators=300

/opt/anaconda3/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [16:05:06] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Validation F1 Score after Tuning: 1.0
Validation ROC AUC Score: 1.0
